In [ ]:
from oneatlas import OneAtlasClient  # custom class in this repo
import json
import geopandas as gpd
from shapely.geometry import box
from pathlib import Path

# Process overview
- Convert input aoi layer into geojson polygons
- For each aoi polygon feature:
  - Search API for available images
  - Plot the quicklook for each image found for that feature
  - Place an order for the chosen image clipped to the feature
  - Download the order once completed

In [ ]:
# Read local config.json to get api key and directory for outputs
with open("config.json", "r") as file:
    config = json.load(file)

api_key = config["api_key"]

client = OneAtlasClient(api_key=api_key)

output_folder = Path(config["output_dir"])
input_file_gdb = Path(config["input_gdb"])

In [ ]:
# function to convert polygon gdf to bounding box list
sites_gdf = gpd.read_file(input_file_gdb, layer="Registered_Sites_Merged_v2")

LAYER_NAME = "Registered_Sites"  # prefix name for the clipped image downloads

In [ ]:
# Drop rows with 'POINT EMPTY' geometry
sites_gdf = sites_gdf[~sites_gdf.geometry.is_empty]

In [ ]:
# Define function to convert points -> buffer -> bounding box
def points_to_buffer_box(gdf, buffer_distance=500):
    """Buffer points by distance and convert to bounding boxes"""

    # Check if all geometries are Points

    if not all(gdf.geometry.geom_type == "Point"):

        print("All geometries in the GeoDataFrame must be Points. Exiting.")

        return None


    # Reproject to EPSG 27700

    gdf = gdf.to_crs(epsg=27700)


    # Buffer the geometries

    gdf["geometry"] = gdf.geometry.buffer(buffer_distance)


    # Convert buffers to bounding boxes

    gdf["geometry"] = gdf.geometry.apply(lambda geom: box(*geom.bounds))

    return gdf

In [ ]:
# convert points geometry to bounding box around 500m buffer
sites_box_gdf = points_to_buffer_box(sites_gdf, buffer_distance=500)

In [ ]:
def aoi_gdf_to_search_geojson(gdf, uid_column=None):
    """Convert geodataframe to geojson ensuring unique ids per feature"""

    if uid_column is None:

        uid_column = "id"

        gdf[uid_column] = gdf.index

    else:

        if not gdf[uid_column].is_unique:

            raise ValueError(
                f"Values in '{uid_column}' are not unique. Consider using default uid_column=None to add uids."
            )

    print(
        f"uid_column '{uid_column}' values {', '.join(str(i) for i in gdf[uid_column].tolist())}"
    )

    json_str = gdf[[uid_column, "geometry"]].to_crs(epsg=4326).to_json(drop_id=True)

    return json.loads(json_str), gdf[uid_column].tolist()

In [ ]:
# convert gdf to geojson and get the id value list to know ids to search for
search_geojson, id_vals = aoi_gdf_to_search_geojson(
    sites_box_gdf, uid_column="image_id"
)

In [ ]:
def get_feature_by_id(geojson, id_vals, search_id, uid_column="id"):
    """Filter geojson features to just one specified id"""

    if search_id not in id_vals:

        raise ValueError(f"id {search_id} not in list of id values")

    for feature in geojson["features"]:

        if feature["properties"][uid_column] == search_id:
            return feature

    return None

In [ ]:
# Set the search id. Must be in the id_vals list created above.
SEARCH_ID = 3
# extract the geojson feature for that id
search_feature = get_feature_by_id(
    search_geojson, id_vals, SEARCH_ID, uid_column="image_id"
)

Search options are described [here](https://www.geoapi-airbusds.com/guides/oneatlas-data/g-search/)

In [ ]:
# create search json - geometry is the search feature geometry
img_search_json = {
    "cloudCover": "[0,30]",
    "incidenceAngle": "[0,40]",
    "processingLevel": "SENSOR",
    "relation": "contains",
    "geometry": search_feature["geometry"],
    "constellation": "PHR",
}

# make the request
results = client.search(img_search_json)

# extract relevant values from the results and store in client instance


client.extract_results(results)

In [ ]:
# Run this cell repeatedly to show each search result image quicklook in turn
client.show_result()

See the order options [here](https://www.geoapi-airbusds.com/guides/oneatlas-data/g-order-product/)

In [ ]:
# create the order specification and initially just quote the price
order_body = {
    "kind": "order.data.product",
    "products": [
        {
            "productType": "pansharpened",  # pansharpened # multiSpectral
            "radiometricProcessing": "DISPLAY",  # REFLECTANCE # DISPLAY #
            "imageFormat": "image/geotiff",
            "crsCode": "urn:ogc:def:crs:EPSG::4326",
            "id": client.current_image,  # current client.show_result() image
            "aoi": search_feature["geometry"],
        }
    ],
}


client.get_price(order_body)["price"]

In [ ]:
# Add a ref to identify the order
order_ref = f"sg_{LAYER_NAME}_{SEARCH_ID}"

order_body["customerRef"] = order_ref

In [ ]:
# Uncomment line below to place the order above (WARNING: account will be charged)
client.create_order(order_body)

In [ ]:
# See order status - need to keep checking by running this until shows "delivered"
orders = client.list_orders(customerRef=order_ref)
order = orders["items"][0]
order["status"]

In [ ]:
# The config.json in the repo specifies the output_folder (I'm using _PS if pan-sharpened)
output_file = output_folder / f"{LAYER_NAME}_{SEARCH_ID}_PS.zip"

# Download the order to specified zip file
client.download_order_to_file(order, output_file)